In [5]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import timedelta
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [6]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [7]:
### choose if you want to do only the 45 validation glaciers
validation_only = 1

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values


print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

45


In [9]:
c = 0
for i in range(len(rgis_to_analyze)):
#     if c>0: continue
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]
    if ga<500: continue
        
    # choose how much to coarsen (more coarse for bigger glaciers) 
    if ga>1000:
        scale=5
    elif ga>500:
        scale=3
    else:
        scale=1
    
    # set folder
    if validation_only:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    else:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
       
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry

    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    
    # coarsen if this is a big glacier
    if ga>500:
        glacier_mask = glacier_mask.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True).astype('uint8')
    
    # count pixels
    glacier_pixels = glacier_mask.sum().values
    
    # open the observed faction df
#     obs_path = os.path.join(folder_save, 'Daily AAs', 'observed', f"S2_{rgiid}_observed.csv")
#     obs_df = pd.read_csv(obs_path)
    
    # create new obs_df
    all_dates = []
    all_obs = []
    
    # for each year, open the daily data, count usable, save to df
    for y in [2018,2019,2020,2021,2022]:
        if ga>100: print(y)
        
        # open
        if ga>500:
            path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed_coarse.nc")
#             path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed.nc")
#             path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs.nc")
        else:
            path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed.nc")
        
        snow = xr.open_dataset(path, chunks={'band':1, 'y':-1, 'x':-1}).to_array()
        
        # get dates
        dates = snow.time.values
        dates = [str(d)[:10] for d in dates]
        
        # count usable fraction each date
        percent_usable_by_time = (xr.where(snow>0, 1, 0).sum(dim=['x','y'])).compute().values[0][0]
        
        # scale correctly
        percent_usable_by_time = percent_usable_by_time/glacier_pixels #*scale*scale
        
        all_dates.append(dates)
        all_obs.append(percent_usable_by_time.round(4))
        
    # flatten out lists    
    all_dates = [i for sl in all_dates for i in sl]
    all_obs = [i for sl in all_obs for i in sl]
#     all_obs = [i for i in all_obs]
    
    # format df and save
    obs_path = os.path.join(folder_save, 'Daily AAs', 'observed', f"S2_{rgiid}_observed.csv")
    obs_df = pd.DataFrame({'Date':all_dates, 'observed_initial':all_obs, 'terrain_shadows':[0 for i in all_obs]})
    obs_df.to_csv(obs_path, index=False)
    c+=1
    
print("Done!")


Starting 45 of 45: RGI60-01.01390  521.396 km2
2018
2019
2020
2021
2022
Done!


In [7]:
all_dates

['2018-05-02',
 '2018-05-04',
 '2018-05-07',
 '2018-05-09',
 '2018-05-12',
 '2018-05-14',
 '2018-05-17',
 '2018-05-19',
 '2018-05-22',
 '2018-05-24',
 '2018-05-27',
 '2018-05-29',
 '2018-06-01',
 '2018-06-03',
 '2018-06-06',
 '2018-06-08',
 '2018-06-11',
 '2018-06-13',
 '2018-06-16',
 '2018-06-18',
 '2018-06-21',
 '2018-06-23',
 '2018-06-26',
 '2018-06-28',
 '2018-07-01',
 '2018-07-03',
 '2018-07-06',
 '2018-07-08',
 '2018-07-11',
 '2018-07-13',
 '2018-07-16',
 '2018-07-18',
 '2018-07-21',
 '2018-07-23',
 '2018-07-26',
 '2018-07-28',
 '2018-07-31',
 '2018-08-02',
 '2018-08-05',
 '2018-08-07',
 '2018-08-10',
 '2018-08-12',
 '2018-08-15',
 '2018-08-17',
 '2018-08-20',
 '2018-08-22',
 '2018-08-25',
 '2018-08-27',
 '2018-08-30',
 '2018-09-01',
 '2018-09-04',
 '2018-09-06',
 '2018-09-09',
 '2018-09-11',
 '2018-09-14',
 '2018-09-16',
 '2018-09-19',
 '2018-09-21',
 '2018-09-24',
 '2018-09-26',
 '2018-09-29',
 '2018-10-01',
 '2018-10-04',
 '2018-10-06',
 '2018-10-09',
 '2018-10-11',
 '2018-10-

In [8]:
all_obs

[0.2819, 1.7666, 3.5325, 0.1116, 0.0]